In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Colab Notebooks/pygaggle-master/

/content/drive/MyDrive/Colab Notebooks/pygaggle-master


In [3]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [4]:
ls

data/
docs/
environment.yml
experiments/
hs_err_pid2944.log
indexes/
LICENSE
logs/
MANIFEST.IN
models/
notebooks/
NQ_dev_top100_qe_reranked_t5_1.json
NQ_dev_top100_qe_reranked_t5_2.json
NQ_dev_top100_qe_reranked_t5_test1.json
NQ_dev_top100_qe_reranked_t5_test2.json
NQ_dev_top100_qe_reranked_t5_test3.json
NQ_dev_top100_qe_reranked_t5_test4.json
NQ_dev_top100_reranked_t5_1.json
NQ_dev_top100_reranked_t5_2.json
NQ_dev_top100_reranked_t5_test5.json
NQ_dev_top100_reranked_t5_test6.json
NQ_dev_top100_reranked_t5_test8.json
NQ_dev_top100_reranked_t5_test9.json
NQ_dev_top100_reranked_tct_colbert-v2-hnp-msmarco-r2.json
NQ_test_benchmark_final.json
NQ_test_benchmark_final_reranked_original.json
NQ_test_benchmark_final_reranked_qe1.json
NQ_test_benchmark_final_reranked_qe2.json
NQ_test_benchmark_final_reranked_t5_originalq.json
NQ_test_benchmark_final_reranked_t5_qe1.json
NQ_test_benchmark_final_reranked_t5_qe2.json
NQ_test_benchmark_reranked_bert_originalq.json
NQ_test_benchmark_reranked_bert_qe

In [5]:
pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 111.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 94.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.7/508.7 kB 30.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 60.1 MB/s eta 0:00:00
     ━━━━━━

In [7]:
cd /content/drive/MyDrive/Colab Notebooks/

/content/drive/MyDrive/Colab Notebooks


In [ ]:
'''
import json
import os

# List the JSON files to be combined
json_files = ['NQ_test_benchmark_1.edited.json', 'NQ_test_benchmark_2_edited_new.json', 'NQ_test_benchmark_3_edited_new.json','NQ_test_benchmark_4_edited (1).json']

# Create an empty list to hold the combined data
combined_data = []

# Read and combine the JSON files
for file_name in json_files:
    with open(file_name, 'r') as file:
        json_data = json.load(file)
        print(len(json_data))
        combined_data.extend(json_data)

# Save the combined data to a new JSON file
output_file = 'NQ_test_benchmark_final_new.json'
with open(output_file, 'w') as file:
    json.dump(combined_data, file)
'''

80
92
59
88


In [8]:
# Metric Functions

import pandas as pd
import numpy as np

def calculate_accuracy(df_json, k=100):
  for i in range(len(df_json)): # iterate per row
    true_cnt = 0
    true_k_cnt = 0
    for j in range(len(df_json[i]['ctxs'])): # iterate per ctxs
      if df_json[i]['ctxs'][j]['has_answer']:
        if j <= k:
          true_k_cnt += 1
        true_cnt += 1
  return true_k_cnt / true_cnt

def calculate_accuracy2(df_json, k=100):
  true_cnt = 0
  true_k_cnt = 0
  for i in range(len(df_json)): # iterate per row
    flag=0  
    for j in range(len(df_json[i]['ctxs'])): # iterate per ctxs
      if df_json[i]['ctxs'][j]['has_answer']:
        flag=1
        true_cnt += 1
        break

    if flag==1:
      for m in range(k):
        if df_json[i]['ctxs'][m]['has_answer']:
          true_k_cnt += 1
          break
  return true_k_cnt / true_cnt

def calculate_precision(df_json, k=100):
  precision = 0
  for i in range(len(df_json)): # iterate per row
    true_cnt = 0
    pos_lst = [1 for x in df_json[i]['ctxs'] if x['has_answer']]
    for j in range(k): # iterate per ctxs
      if df_json[i]['ctxs'][j]['has_answer']:
        true_cnt += 1
    if len(pos_lst):
      curr_precision = true_cnt / len(pos_lst)
    else:
      curr_precision = 0
    precision += curr_precision
  return precision / len(df_json)

def calculate_mrr(df_json, k=100):
  mrr = 0
  for i in range(len(df_json)): # iterate per row
    for j in range(k): # iterate per ctxs
      if df_json[i]['ctxs'][j]['has_answer']:
        mrr += 1 / (j + 1) # (j+1)-th rank
        break
  return mrr / len(df_json)

def calculate_map(df_json, k=100):
  map = 0
  for i in range(len(df_json)): # iterate per row
    true_cnt = 0
    curr_precision = 0
    for j in range(k): # iterate per ctxs
      if df_json[i]['ctxs'][j]['has_answer']:
        true_cnt += 1
        curr_precision += true_cnt / (j + 1)
    if true_cnt:
      map += curr_precision / true_cnt
  return map / len(df_json)

def calculate_ndcg(df_json, k=100):
  ndcg = 0
  for i in range(len(df_json)): # iterate per row
    dcg = 0
    true_cnt = 0
    ideal_dcg = 1
    for j in range(k): # iterate per ctxs
      if df_json[i]['ctxs'][j]['has_answer']:
        dcg += 1 / np.log2(j + 2)
        ideal_dcg += 1 / np.log2(true_cnt + 2)
        true_cnt += 1
    if ideal_dcg or dcg:
      ndcg += dcg / ideal_dcg
  return ndcg / len(df_json)

In [28]:
import json

def reading_json_file(path):
    queryy=[]

    with open(path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        print(len(data))
        corpus = [[['a' for _ in range(4)] for _ in range(100)] for _ in range(len(data))]
        for i in range(len(data)):
            #queryy.append(data[i]["question"])
            #queryy.append(data[i]["expanded_query"])
            #queryy.append(data[i]["expanded_query_2"])
            #queryy.append(data[i]["expanded_query_gpt"])
            queryy.append(data[i]["expanded_query_3"])
            #queryy.append(data[i]["expanded_query_4"])
            for j in range(100):
                corpus[i][j][0]=data[i]["ctxs"][j]["id"]
                corpus[i][j][1]=data[i]["ctxs"][j]["title"]
                corpus[i][j][2]=data[i]["ctxs"][j]["text"]
                corpus[i][j][3]=data[i]["ctxs"][j]["has_answer"]

    print(queryy)
    return queryy, corpus, data

In [17]:
cd /content/drive/MyDrive/Colab Notebooks/pygaggle-master/

/content/drive/MyDrive/Colab Notebooks/pygaggle-master


In [22]:
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoBERT, MonoT5

##reranker 설정 필요
reranker =  MonoBERT()
#reranker =  MonoT5()
     

In [29]:
##path 설정 필요
#file_path ='/content/drive/MyDrive/Colab Notebooks/pygaggle-master/NQ_test_benchmark_final_reranked_original.json'
#file_path ='/content/drive/MyDrive/Colab Notebooks/pygaggle-master/NQ_test_benchmark_final_reranked_qe1.json'
#file_path ='/content/drive/MyDrive/Colab Notebooks/pygaggle-master/NQ_test_benchmark_final_reranked_qe2.json'
#file_path ='/content/drive/MyDrive/Colab Notebooks/NQ_test_benchmark_final_everything.json'
#file_path='/content/drive/MyDrive/Colab Notebooks/pygaggle-master/NQ_test_benchmark_final_reranked_t5_originalq.json'
#file_path='/content/drive/MyDrive/Colab Notebooks/NQ_test_benchmark_withgpt_part_1.json'
file_path='/content/drive/MyDrive/Colab Notebooks/NQ_test_benchmark_withgpt_deduplicated_final.json'

queryy, corpus, data = reading_json_file(file_path)

272
['The southwest monsoon blow over Nigeria during summer\nthe south west wind blows across nigeria between', 'HP stands for High Performance Computing (HPC) which refers to a computer architecture that can perform complex calculations very quickly\nwhat does hp mean in war and order', '### expansion points - Reading Football Club means a team that playes in England Premier League\nwho is the owner of reading football club', 'The earliest known eyes were found on a fossil called "Euthycrania" which lived about 400 million years ago (in the Devonian Period). It had simple light-detecting cells that may have been used for sensing light to help it hunt for food\nwhat is the first step in the evolution of the eye', '### expansion points - Gallbladder means a small organ that sits under your liver on the right side of your abdomen\nwhere is gall bladder situated in human body', 'oil rich region,gas-rich area\nin which regions are most of africa petroleum and natural gas found', 'oil produ

In [30]:
for q in range(len(data)): ##range 설정 필요
  print(q)
  query = Query(queryy[q])
  passages=corpus[q]

  texts = [ Text(p[2], {'id': p[0],'title':p[1],'text':p[2],'has_answer' : p[3]}, 0) for p in passages]

  reranked = reranker.rerank(query, texts)

  #json 파일 값 변경
  for k in range(100):
    data[q]["ctxs"][k]["id"]=reranked[k].metadata["id"]
    data[q]["ctxs"][k]["title"]=reranked[k].metadata["title"]
    data[q]["ctxs"][k]["text"]=reranked[k].metadata["text"]
    data[q]["ctxs"][k]["score"]=str(reranked[k].score) 
    data[q]["ctxs"][k]["has_answer"]=reranked[k].metadata["has_answer"]

k = 10

print(f'Accuracy@{k}: {calculate_accuracy2(data, k):.4f}')
print(f'Precision@{k}: {calculate_precision(data, k):.4f}')
print(f'MRR@{k}: {calculate_mrr(data, k):.4f}')
print(f'MAP@{k}: {calculate_map(data, k):.4f}')
print(f'nDCG@{k}: {calculate_ndcg(data, k):.4f}')

## json 파일명 변경 및 data range 설정 필요
with open('NQ_test_benchmark_reranked_t5_qe3.json', 'w') as file:
      json.dump(data, file)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
Accuracy@10: 0.7831
Pr

In [31]:
k = 5

print(f'Accuracy@{k}: {calculate_accuracy2(data, k):.4f}')
print(f'Precision@{k}: {calculate_precision(data, k):.4f}')
#print(f'Precision2@{k}: {calculate_precision2(data, k):.4f}')
print(f'MRR@{k}: {calculate_mrr(data, k):.4f}')
print(f'MAP@{k}: {calculate_map(data, k):.4f}')
print(f'nDCG@{k}: {calculate_ndcg(data, k):.4f}')

Accuracy@5: 0.6360
Precision@5: 0.2759
MRR@5: 0.4502
MAP@5: 0.4301
nDCG@5: 0.2938


In [32]:
k = 1

print(f'Accuracy@{k}: {calculate_accuracy2(data, k):.4f}')
print(f'Precision@{k}: {calculate_precision(data, k):.4f}')
#print(f'Precision2@{k}: {calculate_precision2(data, k):.4f}')
print(f'MRR@{k}: {calculate_mrr(data, k):.4f}')
print(f'MAP@{k}: {calculate_map(data, k):.4f}')
print(f'nDCG@{k}: {calculate_ndcg(data, k):.4f}')

Accuracy@1: 0.3456
Precision@1: 0.0895
MRR@1: 0.3456
MAP@1: 0.3456
nDCG@1: 0.1728
